<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/gpts/DIY_AK_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn
import torch.functional as F

In [4]:
#####################
# DATASET RETRIEVAL #
#####################

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-20 14:03:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-09-20 14:03:56 (22.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open("input.txt", mode="r") as f:
  text = f.read()

print(text[:10])

First Citi


In [33]:
###################
# HYPERPARAMETERS #
###################

In [46]:
block_size = 8
batch_size = 32
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
###############################
# TOKENIZATION AND DATALOADER #
###############################

In [22]:
vocab = list(sorted(set(text)))
text_to_num = {v:k for k,v in enumerate(vocab)}
num_to_text = {k:v for k,v in enumerate(vocab)}
vocab_size = len(vocab)

In [23]:
encode = lambda text: [text_to_num[t] for t in text]
decode = lambda numbers: "".join([num_to_text[n] for n in numbers])

In [25]:
decode(encode("Haubi"))

'Haubi'

In [37]:
data = torch.tensor(encode(text))
train_idx = int(0.9 * len(data))
train = data[:train_idx]
test = data[train_idx:]
len(train), len(test)

(1003854, 111540)

In [48]:
def get_batch(dataset):
  dataset = train if dataset == "train" else test
  idx = torch.randint(0, len(train) - block_size, (batch_size,))
  X = torch.stack([train[i:i +  block_size] for i in idx])
  Y = torch.stack([train[i + 1: i + block_size + 1] for i in idx])
  X, Y = X.to(device), Y.to(device)

  return X, Y

In [50]:
X, Y = get_batch("train")
X[1], Y[1]

(tensor([58, 46, 47, 57,  1, 57, 59, 40]),
 tensor([46, 47, 57,  1, 57, 59, 40, 51]))